### Config

In [8]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from data_processing import process_and_chunk_data, chunk_single_file
from groq import Groq
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.schema import Document
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.tavily_search import TavilySearchResults
from pprint import pprint
from typing_extensions import TypedDict
from typing import List
import os 
import time

### Embedding & call llm

In [9]:
%pip install fastembed

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [11]:
import os
groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(model_name='Llama3-8b-8192', api_key=groq_api_key)

In [12]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000012105BAA870>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000012105BB1CD0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

###  Chunking text

In [13]:
# raw_texts = process_and_chunk_data('data')
raw_texts = chunk_single_file('data/luat_dat_dai.json')
doc_splits = [Document(page_content=chunk) for chunk in raw_texts]
doc_splits[0]

Document(metadata={}, page_content='Tiêu đề: LUẬT Chương I: QUY ĐỊNH CHUNG Điều 1. Phạm vi điều chỉnh Luật này quy định về chế độ sở hữu đất đai, quyền hạn và trách nhiệm của Nhà nước đại diện chủ sở hữu toàn dân về đất đai và thống nhất quản lý về đất đai, chế độ quản lý và sử dụng đất đai, quyền và nghĩa vụ của công dân, người sử dụng đất đối với đất đai thuộc lãnh thổ của nước Cộng hòa xã hội chủ nghĩa Việt Nam. Điều 2. Đối tượng áp dụng 1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất')

In [14]:
doc_count = len(doc_splits)
doc_count

11318

## RAG

### save vector db in persist files

In [15]:
persist_directory = 'real_estate_db/luat_dat_dai'

### chromadb

In [ ]:
# create db
vectorstore_created = Chroma.from_documents(documents=doc_splits,
                                    embedding=embed_model,
                                    persist_directory=persist_directory,
                                    collection_name="local-rag")

In [ ]:
vectorstore_created.persist()

In [37]:
# # call from existed db
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embed_model)

In [ ]:
print("Number of stored documents:", vectorstore._collection.count())
print("First document:", doc_splits[0].page_content if doc_splits else "No documents found!")

Number of stored documents: 0
First document: Tiêu đề: LUẬT Chương I: QUY ĐỊNH CHUNG Điều 1. Phạm vi điều chỉnh Luật này quy định về chế độ sở hữu đất đai, quyền hạn và trách nhiệm của Nhà nước đại diện chủ sở hữu toàn dân về đất đai và thống nhất quản lý về đất đai, chế độ quản lý và sử dụng đất đai, quyền và nghĩa vụ của công dân, người sử dụng đất đối với đất đai thuộc lãnh thổ của nước Cộng hòa xã hội chủ nghĩa Việt Nam. Điều 2. Đối tượng áp dụng 1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất


In [38]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_lambda = RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"]))

In [19]:
# from langchain.schema import Document
# from langchain_core.runnables import RunnableLambda

# # convert chunks to document
# def format_text_chunks(text_chunks):
#     return [Document(page_content=chunk) for chunk in text_chunks]

# retriever_lambda = RunnableLambda(lambda x: format_text_chunks(retriever.get_relevant_documents(x["question"])))

### Router

In [39]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on Land law and housing law in Vietnam. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)
start = time.time()
question_router = router_prompt | llm | JsonOutputParser()

# test
question = "người dân có quyền sử dụng đất bao nhiêu năm?"
print(question_router.invoke({"question": question}))
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:0.3053898811340332


### Generator

In [40]:
from langchain_core.runnables import RunnableLambda

qa_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just refuse answer in polite and friendly. 
    Use three sentences maximum and keep the answer concise and answer using Vietnamese <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Chain
start = time.time()
rag_chain = (
    {"question": lambda x: x["question"], "context": retriever_lambda}
    | qa_prompt
    | llm
    | StrOutputParser()
)

# test
# question = "luật nhà ở 2024"
response = rag_chain.invoke({"question": question})
print(response)

Theo Luật Đất đai 2013, người dân có quyền sử dụng đất ổn định trong 50 năm. Sau 50 năm, người dân có thể được cấp giấy chứng nhận quyền sử dụng đất lâu dài. Tùy vào từng trường hợp, người dân có thể được quyền sử dụng đất lâu dài hơn.


### Retriever

In [41]:
retrieval_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. \n
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)
start = time.time()
retrieval_grader = retrieval_grader_prompt | llm | JsonOutputParser()

# test
# question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
end = time.time()
print(f"The time required to generate response by the retrieval grader in seconds:{end - start}")


IndexError: list index out of range

In [23]:
docs

[]

### Hallucination

In [24]:
hallucination_grader_prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)
start = time.time()
hallucination_grader = hallucination_grader_prompt | llm | JsonOutputParser()

# test
hallucination_grader_response = hallucination_grader.invoke({"documents": docs, "generation": response})
end = time.time()
print(f"The time required to generate response by the generation chain in seconds:{end - start}")
print(hallucination_grader_response)

The time required to generate response by the generation chain in seconds:0.5395851135253906
{'score': 'yes'}


### ANswer grader

In [ ]:
answer_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = answer_grader_prompt | llm | JsonOutputParser()

# test
answer_grader_response = answer_grader.invoke({"question": question,"generation": response})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:0.23734021186828613
{'score': 'no'}


### Websearch tool

In [ ]:
tavily_api_key = os.environ['TAVILY_API_KEY']
web_search_tool = TavilySearchResults(k=3)


Single test web_search_tool

In [ ]:
import os
from langchain.tools import TavilySearchResults

def search_web(query: str, k: int = 3):
    """
    Searches the web using Tavily API for the given query.
    
    Parameters:
    - query (str): The search query.
    - k (int): Number of search results to return (default is 3).
    
    Returns:
    - list: A list of search result snippets.
    """
    tavily_api_key = os.environ['TAVILY_API_KEY']
    web_search_tool = TavilySearchResults(k=k)
    
    try:
        results = web_search_tool.run(query)
        return results
    except Exception as e:
        print(f"Error while searching: {e}")
        return []

query = "TÌnh hình bất đọngo sản 2024"
search_results = search_web(query)
print(search_results)


[{'url': 'https://kinhtedothi.vn/nhin-lai-thi-truong-bat-dong-san-nam-2024.html', 'content': 'Tính chung cả năm 2024, toàn thị trường ghi nhận khoảng gần 81.000 sản phẩm chào bán, tăng hơn 40% so với năm 2023. Trong đó, có 65,376 sản phẩ'}, {'url': 'https://baochinhphu.vn/chuyen-dong-tich-cuc-cua-thi-truong-bat-dong-san-trong-quy-iii-2024-102241015110847106.htm', 'content': 'Đại diện Viện nghiên cứu Đánh giá thị trường bất động sản Việt Nam cho biết, quý 3 năm 2024, thị trường BĐS nhà ở tiếp tục ghi nhận nguồn cung đạt mức 22.412 sản phẩm được chào bán trên thị trường, với khoảng 14.750 sản phẩm mở bán mới, giảm 25% so với quý trước, nhưng đã tăng 60% so với cùng kỳ năm 2023. Nghiên cứu về chỉ số giá căn hộ, phản ánh mức biến động giá bán bình quân của các dự án trong tập mẫu 150 dự án được Hội Môi giới bất động sản Việt Nam (VARS) chọn lọc và quan sát cũng cho thấy, tính đến quý 3/2024, giá bán trung bình của cụm mẫu dự án ở thành phố Hà Nội gần tiệm cận mức 60 triệu VNĐ/m2, tăng\xa0 

## LangGraphh

In [ ]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]
    iterations: int

### Nodes

In [ ]:
from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    iterations = state.get("iterations", 0) + 1
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation, "iterations": iterations}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    
    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}
#
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])
    iterations = state.get("iterations", 0) + 1

    # Web search
    docs = web_search_tool.run({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question, "iterations": iterations}

### Condition edges

In [ ]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


In [ ]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


In [ ]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    iterations = state.get("iterations", 0)
    
    if iterations >= 10:
        print("GETTING MAX ATTEMPTS")
        return "end"

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

### Add node

In [ ]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

### Entry & End points

In [ ]:
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
        "end": END
    },
)

In [ ]:
app = workflow.compile()

## Test

In [ ]:
from pprint import pprint

inputs = {"question": "Tôi là ai?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
    # if "generation" in value:
    #     pprint(value["generation"])
    # else:
    #     pprint("Hiện mình chưa có thông tin về câu hỏi của bạn. Bạn có thể thử lại với câu hỏi khác không?")
pprint(value["generation"])

---ROUTE QUESTION---
Tôi là ai?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('Tôi không thể trả lời câu hỏi "Tôi là ai?" vì không có thông tin liên quan '
 'đến người nào trong đoạn văn. Xin lỗi!')
